# 0.0 Imports

In [1]:
import pandas as pd

## 0.1 Helper Functions

### Set Display

## 0.2 Load Data

## 0.3 Train & Test Split

# 1.0 Data Description

## 1.1 Data Dimensions

## 1.2 Descriptive Statistics

### Numerical

### Categorical

# 2.0 Feature Engineering

## 2.1 Rename Columns

## 2.2 Feature Engineering

# 3.0 Data Filtering

# 4.0 EDA

## 4.1 Hypothesis Map

## 4.2 Univariate Analysis

# 5.0 Data Preparation

### Standardization, Rescaling and Encoding

# 6.0 Feature Selection

## 6.1 Split Train & Validation

## 6.2 Fature Selection

### Boruta

### Fature Importance

# 7.0 Machine Learnign Modeling

## 7.1 Model Test

## 7.3 Performance Comparation

### Models Results

## 7.4 Cross Validation

# 8.0 Fine Tunning

## 8.1 Random Search

# 9.0 Final Model

## Model Training

### Model Validation

## Final Model

# 10.0 Results Interpretation

# 11.0 Deploy